In [2]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [3]:
bestlcs=pd.read_csv('makingtable/bestlcs.csv')
gaiatable=pd.read_csv('makingtable/cleanedgaiadata.csv')
id_table=pd.read_csv('makingtable/idswithdiscopaper.csv')

In [4]:
bestlclist=np.array([], dtype=object)
start_col=bestlcs.columns.get_loc('Sector 1')
for index, row in bestlcs.iterrows(): #iterate through each object
    for col in bestlcs.columns[start_col:]: #iterate through all the columns starting with the sectors
        value=row[col]
        if not pd.isna(value):
            bestlclist=np.append(bestlclist, [f'{row['pop_id']}_{value}'])

In [5]:
#now doing K2SFF lcs
K2SFFlist=np.loadtxt('makingtable/K2SFFlist.txt', dtype=str)
lclist=pd.concat([pd.DataFrame(bestlclist), pd.DataFrame(K2SFFlist)], ignore_index=True)
lclist[['pop_id', 'lcname']]=lclist[0].str.split('_', expand=True)
lclist.drop(columns=0, inplace=True)

In [6]:
lclist[['idtype', 'ID', 'secorcamp', 'author', 'cadence']] = lclist['lcname'].str.split('-', n=4, expand=True)

In [7]:
lclist

,pop_id,lcname,idtype,ID,secorcamp,author,cadence
0,0,TIC-59129133-5-FFI-30min,TIC,59129133,5,FFI,30min
1,0,TIC-59129133-32-FFI-10min,TIC,59129133,32,FFI,10min
2,0,TIC-59129133-43-SPOC-120,TIC,59129133,43,SPOC,120
3,0,TIC-59129133-44-SPOC-120,TIC,59129133,44,SPOC,120
4,0,TIC-59129133-71-FFI-200s,TIC,59129133,71,FFI,200s
...,...,...,...,...,...,...,...
979,201,EPIC-60019763-1-K2SFF-1800,EPIC,60019763,1,K2SFF,1800
980,208,EPIC-246776923-13-K2SFF-1800,EPIC,246776923,13,K2SFF,1800
981,209,EPIC-246938594-13-K2SFF-1800,EPIC,246938594,13,K2SFF,1800
982,210,EPIC-246969828-13-K2SFF-1800,EPIC,246969828,13,K2SFF,1800


In [16]:
for index, currow in lclist.iterrows():
    if pd.isna(currow['fluxamplitude']):
        try:
            curlc=get_lc_string(currow['lcname'])
            if currow['author']=='FFI':
                curlc.flux=curlc.flux+1 #shift normalization to be around 1
            else:
                curlc=curlc.normalize() #normalize once now instead of having to do it in every method
            ninety=np.percentile(curlc['flux'],90)
            ten=np.percentile(curlc['flux'],10)
            amplitude=ninety-ten
            lclist.at[index, 'fluxamplitude']=amplitude
        except Exception as e:
            print(f'Error processing {currow['lcname']}: {e}')

Error processing EPIC-60019763-1-K2SFF-1800: 'NoneType' object has no attribute 'normalize'


In [18]:
lclist

,pop_id,lcname,idtype,ID,secorcamp,author,cadence,fluxamplitude
0,0,TIC-59129133-5-FFI-30min,TIC,59129133,5,FFI,30min,0.041935
1,0,TIC-59129133-32-FFI-10min,TIC,59129133,32,FFI,10min,0.053793
2,0,TIC-59129133-43-SPOC-120,TIC,59129133,43,SPOC,120,0.053993
3,0,TIC-59129133-44-SPOC-120,TIC,59129133,44,SPOC,120,0.056758
4,0,TIC-59129133-71-FFI-200s,TIC,59129133,71,FFI,200s,0.049836
...,...,...,...,...,...,...,...,...
979,201,EPIC-60019763-1-K2SFF-1800,EPIC,60019763,1,K2SFF,1800,NaN
980,208,EPIC-246776923-13-K2SFF-1800,EPIC,246776923,13,K2SFF,1800,0.059540
981,209,EPIC-246938594-13-K2SFF-1800,EPIC,246938594,13,K2SFF,1800,0.030775
982,210,EPIC-246969828-13-K2SFF-1800,EPIC,246969828,13,K2SFF,1800,0.089235


In [21]:
lclist.to_csv('amplitudetable.csv',index=False)